# Regressão Linear

In [ ]:
!pip install kagglehub

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "lespin/house-prices-dataset",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# 1. Verificando valores nulos
# nulls = df.isnull().sum()
# null_cols = nulls[nulls > 0].index.tolist()
# print("Colunas com valores nulos:", null_cols)

# 2. Remover colunas com muitos nulos e preencher outras
df.fillna(df.median(numeric_only=True), inplace=True)
df.fillna("None", inplace=True)

# 3. Codificação das variáveis categóricas
cat_cols = df.select_dtypes(include=["object"]).columns
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# 4. Separar variáveis preditoras e alvo
X = df.drop(columns=["SalePrice", "Id"])
y = df["SalePrice"]

# 5. Separação treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

# 6. Modelo
model = LinearRegression()
model.fit(X_train, y_train)

# 7. Previsões
y_pred = model.predict(X_test)

# 8. Métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.4f}")

# 9. Gráficos
plt.figure(figsize=(18, 5))

# Gráfico de dispersão
plt.subplot(1, 3, 1)
sns.scatterplot(x=y_test, y=y_pred, color='lightblue')
plt.xlabel("Preço Real")
plt.ylabel("Preço Predito")
plt.title("Preço Real vs Predito")

# Resíduos
plt.subplot(1, 3, 2)
residuals = y_test - y_pred
sns.histplot(residuals, bins=30, color='lightblue', kde=True)
plt.title("Distribuição dos Resíduos")
plt.xlabel("Resíduo")

# 11. Importância das variáveis
coefs = model.coef_
feature_names = X.columns

coef_df = pd.DataFrame({
    "Variável": feature_names,
    "Coeficiente": coefs,
    "Importância": np.abs(coefs)
}).sort_values(by="Importância", ascending=False)

top_10 = coef_df.head(10)

# Gráfico de importância
plt.subplot(1, 3, 3)
sns.barplot(x="Importância", y="Variável", data=top_10, palette="Blues_d")
plt.title("Top 10 Variáveis Mais Importantes (|coef|)")

plt.tight_layout()
plt.show()


# Random Forest (Floresta Aleatória)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 6. Treinar modelo
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 7. Previsões
y_pred = rf.predict(X_test)

# 8. Avaliação
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n Resultados do RandomForestRegressor:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.4f}")

# 9. Gráfico: Preço real vs predito
plt.figure(figsize=(16, 5))

plt.subplot(1, 3, 1)
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5)
plt.xlabel("Preço Real")
plt.ylabel("Preço Predito")
plt.title("Preço Real vs Predito")

# 10. Gráfico: Distribuição dos resíduos
plt.subplot(1, 3, 2)
residuals = y_test - y_pred
sns.histplot(residuals, bins=30, kde=True)
plt.title("Distribuição dos Resíduos")
plt.xlabel("Erro")

# 11. Importância das variáveis
coefs = model.coef_
feature_names = X.columns

coef_df = pd.DataFrame({
    "Variável": feature_names,
    "Coeficiente": coefs,
    "Importância": np.abs(coefs)
}).sort_values(by="Importância", ascending=False)

top_10 = coef_df.head(10)

# Gráfico de importância
plt.subplot(1, 3, 3)
sns.barplot(x="Importância", y="Variável", data=top_10, palette="Blues_d")
plt.title("Top 10 Variáveis Mais Importantes (|coef|)")

plt.tight_layout()
plt.show()


#XGBoost

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBRegressor, plot_importance

# 5. Treinar modelo XGBoost
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
xgb.fit(X_train, y_train)

# 6. Previsões
y_pred = xgb.predict(X_test)

# 7. Avaliação
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n📊 Resultados do XGBoost:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.4f}")

# 8. Gráficos
plt.figure(figsize=(16, 5))

# Real vs Predito
plt.subplot(1, 3, 1)
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.xlabel("Preço Real")
plt.ylabel("Preço Predito")
plt.title("Preço Real vs Predito (XGBoost)")

# Resíduos
plt.subplot(1, 3, 2)
residuals = y_test - y_pred
sns.histplot(residuals, bins=30, kde=True)
plt.title("Distribuição dos Resíduos")
plt.xlabel("Erro")

# 11. Importância das variáveis
coefs = model.coef_
feature_names = X.columns

coef_df = pd.DataFrame({
    "Variável": feature_names,
    "Coeficiente": coefs,
    "Importância": np.abs(coefs)
}).sort_values(by="Importância", ascending=False)

top_10 = coef_df.head(10)

# Gráfico de importância
plt.subplot(1, 3, 3)
sns.barplot(x="Importância", y="Variável", data=top_10, palette="Blues_d")
plt.title("Top 10 Variáveis Mais Importantes (|coef|)")

plt.tight_layout()
plt.show()

#Comparando desempenhos

In [ ]:
# Dicionário para armazenar as métricas
results = {}

# Função para treinar modelo, prever e calcular métricas
def avaliar_modelo(nome, modelo):
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results[nome] = {"MAE": mae, "MSE": mse, "R²": r2}

# Treinar e avaliar os três modelos
avaliar_modelo("Linear Regression", LinearRegression())
avaliar_modelo("Random Forest", RandomForestRegressor(n_estimators=100, random_state=42))
avaliar_modelo("XGBoost", XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42))

# Converter resultados em DataFrame
results_df = pd.DataFrame(results).T.reset_index().rename(columns={"index": "Modelo"})

# Gráficos de comparação
plt.figure(figsize=(15, 5))

# MAE
plt.subplot(1, 3, 1)
sns.barplot(data=results_df, x="Modelo", y="MAE")
plt.title("MAE por Modelo")
plt.xticks(rotation=15)

# MSE
plt.subplot(1, 3, 2)
sns.barplot(data=results_df, x="Modelo", y="MSE")
plt.title("MSE por Modelo")
plt.xticks(rotation=15)

# R²
plt.subplot(1, 3, 3)
sns.barplot(data=results_df, x="Modelo", y="R²")
plt.title("R² por Modelo")
plt.xticks(rotation=15)

plt.tight_layout()
plt.show()

# Exibir os valores também
print("\n📋 Tabela de métricas comparativas:")
print(results_df.round(2))